# Load the Dogs vs Cats Dataset

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from tensorflow.keras import models

In [2]:
(train_set, valid_set, test_set), info = tfds.load(
    'cats_vs_dogs', 
    # take 50% for training, 25% for validation, and 25% for testing
    split=["train[:50%]", "train[50%:75%]", "train[75%:100%]"],
    as_supervised=True,
    with_info=True,
)

In [3]:
info

In [4]:
class_names = info.features["label"].names
class_names

In [5]:
# number of images in the dataset
info.splits["train"].num_examples

In [6]:
import matplotlib.pyplot as plt

img_size = (150, 150)
plt.figure(figsize=(10, 10))
i = 0
for image, label in train_set.take(12):
    plt.subplot(3, 4, i + 1)
    image = tf.image.resize(image, img_size)
    plt.imshow(image.numpy().astype("uint8"))
    plt.title(class_names[label])
    plt.axis("off")
    i += 1

# Build an Input Pipeline

In [7]:
def create_dataset(ds, batch_size=32, buffer_size=1000, shuffle=True):
    ds = ds.map(
            lambda x, y: (tf.image.resize(x, img_size), y), 
            num_parallel_calls=tf.data.AUTOTUNE
        )
    ds = ds.cache() # cache before shuffling for a better performance.
    if shuffle:
        ds = ds.shuffle(buffer_size, seed=42)
    # batch after shuffling to get unique batches at each epoch
    ds = ds.batch(batch_size) 
    return ds.prefetch(1)

train_set = create_dataset(train_set, shuffle=True)
valid_set = create_dataset(valid_set)
test_set = create_dataset(test_set)

In [8]:
images, labels = next(iter(train_set))
print(images.shape)
print(labels.shape)

In [9]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

In [10]:
# take an image from the batch
image = images[3]

plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1) 
    augmented_image = data_augmentation(image)
    plt.imshow(augmented_image / 255)
    plt.axis("off")

# Feature Extraction

In [11]:
# load the pretrained model
base_model = tf.keras.applications.VGG19(
        input_shape=(150, 150, 3),
        include_top=False,
        weights="imagenet",
)
# freeze all layers of the model
base_model.trainable = False

In [12]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(150, 150, 3)),
    data_augmentation,
    layers.Rescaling(scale=1.0/255), # rescale the pixel values to the [0, 1] range
    base_model,
    # add a new FC classifier on top of the base model
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

In [13]:
# compile and start training after freezing the layers
learning_rate = 1e-4
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
              metrics=['acc']
             )

epochs = 10
history = model.fit(train_set, epochs=epochs,
                   validation_data=valid_set)

In [14]:
def plot_learning_curves():
    acc = history.history['acc']
    val_acc = history.history['val_acc']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(10, 7))
    plt.plot(range(epochs), acc, "b", label="Training Accuracy")
    plt.plot(range(epochs), val_acc, "r", label="Validation Accuracy")
    plt.legend()

    plt.plot(range(epochs), loss, "g", label="Training Loss")
    plt.plot(range(epochs), val_loss, "orange", label="Validation Loss")
    plt.legend()
    plt.grid(True)

    plt.show()

plot_learning_curves()

# Fine-tuning

In [15]:
# we start by unfreezing all layers of the base model
base_model.trainable = True

In [16]:
# Freeze all layers except the 10 last layers 
for layer in base_model.layers[:-10]: 
    layer.trainable = False

# compile and retrain with a low learning rate
low_lr = learning_rate / 10 # low learning rate
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=low_lr), 
              metrics=['acc']
)

epochs = 10
history = model.fit(train_set, epochs=epochs,
                    validation_data=valid_set)

In [17]:
plot_learning_curves()

# Evaluate the Model on the Test Set

In [18]:
import numpy as np


test_loss, test_acc = model.evaluate(test_set)
print(np.round(test_acc * 100, 2), '%')

In [19]:
image_batch, label_batch = next(iter(test_set))
proba = model.predict(image_batch)
# returns 0 if the probability of the prediction
# is below 0.5, otherwise it returns 1
y_preds = tf.where(proba < 0.5, 0, 1)

In [20]:
def show_prediction(image, y_pred):
    plt.imshow(image / 255)
    plt.title(class_names[y_pred.numpy()[0]])
    plt.axis("off")


plt.figure(figsize=(10, 10))
for i in range(12):
    plt.subplot(3, 4, i + 1)
    show_prediction(image_batch[i], y_preds[i])